# Happy or not?

I'll be using transfer learning with keras, to find out if a certain image depicts a happy person or not. There might be better or more efficient approaches to doing this, and maybe this turns out to be overkill. However, I wish to know if this would work well. 
I'll be using pre-trained inception model v3 and add a couple of FC layers at the end to check how well it does. 
For this I have a dataset of about 250 images scrapped off the internet. I would suggest you to not use google since it is biased in the sense you would not get asians or indians pictures in the pool. Bing is better. Best would be instagram. 

In [1]:
import numpy as np
import pandas as pd
import h5py

import os, cv2, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from matplotlib import ticker
%matplotlib inline 

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

# First attempt will be with the inception v3 model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import backend as K

Using TensorFlow backend.
/home/paperspace/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
TRAIN_DIR = './dataset/'
CLASSES = ['happy', 'nothappy']

# Image dimentions for resizing:
ROWS = 299  # Default input of inception v3 
COLS = 299 # Default input of inception v3 
CHANNELS = 3

In [4]:
def get_images(emotion):
    """Load files from train folder"""
    emotion_dir = TRAIN_DIR+'{}'.format(emotion)
    images = [emotion+'/'+im for im in os.listdir(emotion_dir)]
    return images

def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (COLS, ROWS), interpolation=cv2.INTER_CUBIC)
    return im


files = []
y_all = []

for emotion in CLASSES:
    emotion_files = get_images(emotion)
    files.extend(emotion_files)
    
    y_emotion = np.tile(emotion, len(emotion_files))
    y_all.extend(y_emotion)
    print("{0} photos of {1}".format(len(emotion_files), emotion))
    
y_all = np.array(y_all)

256 photos of happy
190 photos of nothappy


In [5]:
X_all = np.ndarray((len(files), ROWS, COLS, CHANNELS), dtype=np.uint8)

In [6]:
for i, im in enumerate(files): 
    X_all[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))

print(X_all.shape)

Processed 0 of 446
(446, 299, 299, 3)


In [7]:
y_all

array(['happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy',
       'happy', 'hap

In [8]:
y_all = LabelEncoder().fit_transform(y_all)

In [9]:
y_all = np_utils.to_categorical(y_all)

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, 
                                                    test_size=0.05, random_state=23, 
                                                    stratify=y_all)

In [20]:
X_train.shape

(423, 299, 299, 3)

In [21]:
y_train.shape

(423, 2)

In [22]:
X_valid.shape

(23, 299, 299, 3)

In [14]:
# Remember that as a part of the preprocessing we have already scaled down the images to fit
# the inception model input requirements.

# create the base pre-trained model from Keras library
base_model = InceptionV3(weights='imagenet', include_top=False) #change weights to 'imagenet' on your local build

In [15]:
import tensorflow as tf
print(tf.__version__)

1.4.0


In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer for the FISH_CLASSES = 8 we are trying to predict
predictions = Dense(len(CLASSES), activation='softmax', name='prediction_layer')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first N layers and unfreeze the rest:

N=172

for layer in model.layers[:N]:
    layer.trainable = False
for layer in model.layers[N:]:
    layer.trainable = True

In [18]:
# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train, batch_size=25, epochs=10,
              validation_split=0.2, verbose=1, shuffle=True)

Train on 338 samples, validate on 85 samples
Epoch 1/30
338/338 [==============================] - 19s - loss: 0.8941 - acc: 0.6746 - val_loss: 7.7746 - val_acc: 0.5176
Epoch 2/30
338/338 [==============================] - 8s - loss: 0.2909 - acc: 0.8817 - val_loss: 7.7299 - val_acc: 0.5176
Epoch 3/30
338/338 [==============================] - 8s - loss: 0.1001 - acc: 0.9615 - val_loss: 5.8976 - val_acc: 0.5176
Epoch 4/30
338/338 [==============================] - 8s - loss: 0.0791 - acc: 0.9763 - val_loss: 7.7746 - val_acc: 0.5176
Epoch 5/30
338/338 [==============================] - 8s - loss: 0.1343 - acc: 0.9408 - val_loss: 7.7265 - val_acc: 0.5059
Epoch 6/30
338/338 [==============================] - 8s - loss: 0.1742 - acc: 0.9349 - val_loss: 7.7746 - val_acc: 0.5176
Epoch 7/30
338/338 [==============================] - 8s - loss: 0.0821 - acc: 0.9763 - val_loss: 3.7482 - val_acc: 0.6588
Epoch 8/30
338/338 [==============================] - 8s - loss: 0.0216 - acc: 0.9941 - val_l

In [24]:
loss_and_metrics = model.evaluate(X_valid, y_valid, batch_size=20)
print ("Validation Logloss: ", loss_and_metrics)

23/23 [==============================] - 1s     
Validation Logloss:  [0.96461069922288123, 0.82608696688776428]


In [25]:
TEST_DIR = './test/'
os.listdir(TEST_DIR)

['.ipynb_checkpoints',
 'happy-group.jpg',
 'sad-baby-crying.jpg',
 'sad-bearded-man.jpg',
 'happy-lady.jpg',
 'sad-kid-crying.jpg',
 'o-david-cameron-sad-facebook.jpg',
 'sad-baby-heartbroken.jpg']

In [26]:

test_files = [im for im in os.listdir(TEST_DIR) if im.endswith('jpg')]
test = np.ndarray((len(test_files), ROWS, COLS, CHANNELS), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)

test.shape

(7, 299, 299, 3)

In [27]:
test_preds = model.predict(test, verbose=1)

7/7 [==============================] - 1s


In [28]:
test_preds

array([[  9.78325188e-01,   2.16747858e-02],
       [  7.65615329e-02,   9.23438430e-01],
       [  9.99880626e-07,   9.99999046e-01],
       [  2.46208856e-05,   9.99975324e-01],
       [  2.58152359e-05,   9.99974132e-01],
       [  2.87356641e-04,   9.99712646e-01],
       [  1.15130991e-01,   8.84869039e-01]], dtype=float32)

In [29]:
submission = pd.DataFrame(test_preds, columns=CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

,image,happy,nothappy
0,happy-group.jpg,9.783252e-01,0.021675
1,sad-baby-crying.jpg,7.656153e-02,0.923438
2,sad-bearded-man.jpg,9.998806e-07,0.999999
3,happy-lady.jpg,2.462089e-05,0.999975
4,sad-kid-crying.jpg,2.581524e-05,0.999974


So, as we see, the results are good. They would be even better if we train for more epochs. :-)

**Let's save the model, so we could deploy the current version**

In [30]:
model.save('./happy-or-not-v1.h5')